In [7]:
# Task 3: Fraud Detection System

#import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import joblib
import os

# Step 2: Load the dataset (adjust path if needed)
csv_path = '/content/creditcard.csv'  # ensure this file is in your working directory

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"File '{csv_path}' not found. Please check the path.")

df = pd.read_csv(csv_path)  # Dataset should contain a 'Class' column

# Step 3: Separate features and target
X = df.drop('Class', axis=1)
y = df['Class']

# Step 4: Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Handle imbalanced data using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Step 6: Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Step 7: Make predictions on test data
y_pred = model.predict(X_test)

# Step 8: Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Step 9: Save model and columns
joblib.dump(model, 'fraud_model.pkl')
joblib.dump(X.columns.tolist(), 'model_columns.pkl')


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.83      0.83        98

    accuracy                           1.00     56962
   macro avg       0.91      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962



['model_columns.pkl']

In [ ]:
import pandas as pd
import joblib

# Load model and column names
model = joblib.load('fraud_model.pkl')
columns = joblib.load('model_columns.pkl')

def predict_transaction(input_data):
    if len(input_data) != len(columns):
        raise ValueError(f"Expected {len(columns)} features, got {len(input_data)}")

    input_df = pd.DataFrame([input_data], columns=columns)
    prediction = model.predict(input_df)
    return "Fraudulent" if prediction[0] == 1 else "Not Fraudulent"

def main():
    print("---- Fraud Detection System ----")
    print(f"Please enter values for {len(columns)} features:")

    input_data = []
    for col in columns:
        while True:
            try:
                value = float(input(f"{col}: "))
                input_data.append(value)
                break
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

    result = predict_transaction(input_data)
    print(f"\nPrediction: {result}")

if __name__ == "__main__":
    main()
